# Run CO3D Sequence (zero parameter)

In [ ]:
import sys, os, glob
import numpy as np
import pandas as pd
from utils_opt import readFlow

import skimage.io as sio
import matplotlib.pyplot as plt

In [ ]:
dataset_dir = 'rvid/teddybear_34_1479_4753/'
co3d_seq = os.path.split(dataset_dir.rstrip('/').lstrip('/'))[-1]
output_folder = os.path.join('tmp_out_zpfm',co3d_seq)
NUM_MIXTURE = 40
shape_scale = 1.8
c_scale = 4.5
f_scale = 210
rand_sphere_size = 55
cov_scale = 1.2e-2
weight_scale = 1.1
LR_RATE = 0.08
Nepoch = 10
batch_size = 50000

## Load Data

In [ ]:
import gzip
import pickle


with gzip.open(os.path.join(dataset_dir,'pose_depth.pkl.gz'),'rb') as fp:
    depth_and_pose = pickle.load(fp)
    
true_depths = depth_and_pose['depths']
fl = depth_and_pose['fl']
poses = np.array(depth_and_pose['poses'])

In [ ]:
masks_folder = os.path.join(dataset_dir,'masks','video1')
in_files = sorted(glob.glob(masks_folder + '/*.png'))

masks = []
for img_loc in in_files:
    mask = sio.imread(img_loc).astype(np.float32)
    mask = (mask > 0).astype(np.float32)
    masks.append(mask)
masks = np.array(masks)
PY,PX = mask.shape
image_size = (PY,PX)

In [ ]:
masks_folder = os.path.join(dataset_dir,'JPEGImages','video1')
in_files = sorted(glob.glob(masks_folder + '/*.jpg'))

images = []
for img_loc in in_files:
    img = sio.imread(img_loc).astype(np.float32)
    images.append(img)
images = np.array(images)

In [ ]:
fwd_flows = []
bwd_flows = []

flow_fol = os.path.join(dataset_dir,'Flow','video1','*.flo')

flow_files = sorted(glob.glob(flow_fol))

for flfile in flow_files:
    new_flow = readFlow(flfile)
    if PY > PX:
        new_flow = np.stack([new_flow[:,:,1],new_flow[:,:,0]],axis=2)
    if 'bwd' in flfile:
        bwd_flows.append(new_flow)
    else:
        fwd_flows.append(new_flow)


# last flow has no fowards
fwd_flows = fwd_flows + [new_flow*0]
# first flow has no backwards
bwd_flows = [new_flow*0] + bwd_flows

In [ ]:
if 'mesh' in depth_and_pose:
    pt_cld = depth_and_pose['mesh'].vertices
    import sklearn.mixture as mixture

    idx2 = np.arange(pt_cld.shape[0])
    np.random.shuffle(idx2)
    clf = mixture.GaussianMixture(40)
    clf.fit(pt_cld[idx2[:10000]])

    pt_cld_shape_scale = float(pt_cld.std(0).mean())*3 
    center = pt_cld.mean(0)
else:
    pt_cld_shape_scale = 3.0
    center = np.zeros(3,dtype=np.float32)

In [ ]:
SCALE_MUL_FACTOR = shape_scale/pt_cld_shape_scale
# gradients can be sensitive to scale. here we solve on scale = 2.2

In [ ]:
import os
#os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import jax
#jax.config.update('jax_platform_name', 'cpu')

import jax.numpy as jnp
import zpfm_render

render_jit = jax.jit(zpfm_render.render_func_idx_quattrans)


In [ ]:
img_shape = (PY,PX)
min_size_idx = np.argmin(img_shape)
min_size = img_shape[min_size_idx]
max_size = img_shape[1-min_size_idx]
invF = 0.5*min_size/fl
min_dim = np.linspace(-1,1,min_size)
aspect = max_size/min_size
max_dim = np.linspace(-aspect,aspect,max_size)
grid = [-max_dim,-min_dim,1,0] if min_size_idx == 0 else [-min_dim,-max_dim,1,0]
pixel_list = np.transpose(np.squeeze(np.meshgrid(*grid,indexing='ij')),(2,1,0))

print(pixel_list.shape,img_shape)
pixel_list = pixel_list.reshape((-1,4))

In [ ]:
poses = jnp.array(poses)

In [ ]:
from util import image_grid

In [ ]:
# random init settings
rand_mean = center+pt_cld_shape_scale*np.random.multivariate_normal(mean=[0,0,0],cov=cov_scale*np.identity(3),size=NUM_MIXTURE)
rand_weight_log = jnp.log(weight_scale*np.ones(NUM_MIXTURE)/NUM_MIXTURE)
rand_prec = jnp.array([np.identity(3)*rand_sphere_size/pt_cld_shape_scale for _ in range(NUM_MIXTURE)])
rand_color = jnp.array(np.random.randn(NUM_MIXTURE,3))

init_alphas = []
for i in range(len(poses)):
    pixel_list[:,3] = i
    res_img,est_alpha,_,_ = render_jit(rand_mean,rand_prec,rand_weight_log,pixel_list,invF,poses)

    res_imgA = np.array(res_img)
    res_imgA[est_alpha < 0.5] = np.nan
    init_alphas.append(est_alpha.reshape((PY,PX)))
image_grid(init_alphas,6,6,rgb=False)

In [ ]:
total_ray_set = []
for i in range(len(poses)):
    pixel_list[:,3] = i

    total_ray_set.append(pixel_list.copy())

In [ ]:
all_rays = jnp.vstack(total_ray_set)
# scaled into ray coord space, vectorized flows
fwv_flow = jnp.array(np.array(fwd_flows).reshape((-1,2)))/(min_size/2)
bwv_flow = jnp.array(np.array(bwd_flows).reshape((-1,2)))/(min_size/2)

In [ ]:
render_jit_ray = jax.jit(zpfm_render.render_func_rays)
last_img_size = np.prod(img_shape)
v_idx = 40

In [ ]:
def objective(params,camera_rays,invF,poses,true_alpha,true_fwd,true_bwd,true_color):
    CLIP_ALPHA = 1e-6
    means,prec,weights_log,colors = params
    est_depth, est_alpha, est_norm, est_w,flowp,flowm = zpfm_render.render_func_idx_quattrans_flow(means,prec,weights_log,camera_rays,invF,poses)
    est_w = est_w.T
    est_w = est_w/jnp.maximum(est_w.sum(axis=1,keepdims=True),1e-7)
    
    est_color = est_w @ (jnp.tanh(colors)*0.5+0.5)
    est_alpha = jnp.clip(est_alpha,CLIP_ALPHA,1-CLIP_ALPHA)
    mask_loss = - ((true_alpha * jnp.log(est_alpha)) + (1-true_alpha)*jnp.log(1-est_alpha))
    pad_alpha = (true_alpha)[:,None]
    flow1 = jnp.abs(pad_alpha*true_fwd-pad_alpha*jnp.nan_to_num(flowp))
    flow2 = jnp.abs(pad_alpha*true_bwd-pad_alpha*jnp.nan_to_num(flowm))
    cdiff = jnp.abs( (true_color-est_color)*true_alpha[:,None] )
    return mask_loss.mean() + c_scale*cdiff.mean() + f_scale*(flow1.mean() + flow2.mean())  #+ 0
grad_render3 = jax.value_and_grad(objective)


In [ ]:
SCALE_MUL_FACTOR

In [ ]:
import optax
from tqdm.notebook import tqdm
from util import DegradeLR

vecM = jnp.array([1,1,1,1,SCALE_MUL_FACTOR,SCALE_MUL_FACTOR,SCALE_MUL_FACTOR])[None]

train_size = all_rays.shape[0]
Niter_epoch = int(round(train_size/batch_size))

def irc(x): return int(round(x))

# babysit learning rates
adjust_lr = DegradeLR(LR_RATE,0.5,irc(Niter_epoch*0.25),irc(Niter_epoch*0.1),-1e-4)

optimizer = optax.adam(adjust_lr.step_func)

tmp = [rand_mean,rand_prec,rand_weight_log,rand_color]
#tmp = [means,prec,weights_log]

opt_state = optimizer.init(tmp)

all_sils = jnp.hstack([_.ravel() for _ in masks]).astype(jnp.float32)
all_colors = jnp.hstack([_.ravel()/255.0 for _ in images]).astype(jnp.float32).reshape((-1,3))
all_colors = all_colors**(1/2.2)

losses = []
opt_configs = []
outer_loop = tqdm(range(Nepoch), desc=" epoch", position=0)

rand_idx = np.arange(train_size)
params = tmp
def inner_iter(j_idx,rand_idx_local,opt_state,p):
    idx = jax.lax.dynamic_slice(rand_idx_local,[j_idx*batch_size],[batch_size])

    val,g = grad_render3([p[0]*SCALE_MUL_FACTOR,p[1]/SCALE_MUL_FACTOR,p[2],p[3]],all_rays[idx],invF,vecM*poses,all_sils[idx],fwv_flow[idx],bwv_flow[idx],all_colors[idx])   
    updates, opt_state = optimizer.update(g, opt_state,p)
    p = optax.apply_updates(p, updates)
    return val, opt_state, p 
jax_iter = jax.jit(inner_iter)
done = False
for i in outer_loop:
    np.random.shuffle(rand_idx)
    rand_idx_jnp = jnp.array(rand_idx)

    for j in tqdm(range(Niter_epoch), desc=" iteration", position=1, leave=False):
        opt_configs.append(list(params))
        val,opt_state,params = jax_iter(j,rand_idx_jnp,opt_state,params)
        val = float(val)
        losses.append(val)
        if np.isnan(val):
            raise

        if adjust_lr.add(val):
            done = True
            break
        outer_loop.set_description(" loss {:.3f}".format(val))
    if done:
        break

In [ ]:
plt.plot(losses)

In [ ]:
final_mean, final_prec, final_weight_log,final_color = params

In [ ]:
result_depths = []
result_alphas = []
results_colors = []

for i in range(len(poses)):
    pixel_list[:,3] = i
    res_img,est_alpha,_,w = render_jit(final_mean, final_prec, final_weight_log,pixel_list,invF,poses)
    est_color = np.array(w.T @ (jnp.tanh(final_color)*0.5+0.5))**(2.2)

    res_imgA = np.array(res_img)
    est_alpha = np.array(est_alpha)
    res_imgA[est_alpha < 0.5] = np.nan
    est_color[est_alpha < 0.5] = np.nan

    result_depths.append(res_imgA.reshape((PY,PX)))
    result_alphas.append(est_alpha.reshape((PY,PX)))
    results_colors.append(est_color.reshape((PY,PX,3)))

In [ ]:
plt.subplot(1,3,1)
plt.imshow(result_alphas[-1])
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(result_depths[-1])
plt.axis('off')
plt.subplot(1,3,3)
plt.imshow(est_color.reshape((PY,PX,3)),interpolation='nearest')
plt.axis('off')


In [ ]:
from scipy.stats import trim_mean
errs = []
d1f = np.hstack([_.ravel() for _ in  true_depths]).ravel()
d2f = np.hstack([_.ravel() for _ in result_depths]).ravel()

mask = (all_sils !=0 ) & (~np.isnan(d1f)) & (~np.isnan(d2f)) & (d1f !=0) 

trim_mean(abs(d1f[mask]-d2f[mask]),0.1)

In [ ]:
image_grid(masks,rows=3,cols=5,rgb=False)

In [ ]:
max_frame = len(poses)
FWD_BCK_TIMES = 4
THRESH_IDX = np.where(np.array(losses)/min(losses) < 1.02)[0][0]
USE_FIRST_N_FRAC = THRESH_IDX/len(losses)
N_FRAMES = max_frame*FWD_BCK_TIMES
opt_to_use = np.round(np.linspace(0,int(np.floor(len(opt_configs)*USE_FIRST_N_FRAC-1)),N_FRAMES)).astype(int)

In [ ]:
THRESH_IDX/len(losses)

In [ ]:
plt.plot(losses[:THRESH_IDX])

In [ ]:
if os.path.exists(output_folder):
    import shutil
    shutil.rmtree(output_folder)
os.makedirs(output_folder, exist_ok=True)

In [ ]:
frame_idxs = []
frame_list = list(range(max_frame))
for i in range(FWD_BCK_TIMES):
    if (i % 2) == 0:
        frame_idxs += frame_list
    else:
        frame_idxs += frame_list[::-1]

In [ ]:
full_res_alpha = []
full_res_depth = []
full_res_color = []

for r_idx,c_idx in zip(frame_idxs,opt_to_use):
    p = opt_configs[c_idx]

    pixel_list[:,3] = r_idx
    est_depth,est_alpha,_,w = render_jit(p[0],p[1],p[2],pixel_list,invF,poses)
    est_color = np.array(w.T @ (jnp.tanh(p[3])*0.5+0.5))**(2.2)

    est_alpha = np.array(est_alpha)
    est_depth = np.array(est_depth)
    est_depth[est_alpha < 0.5] = np.nan
    est_color[est_alpha < 0.5] = np.nan

    full_res_alpha.append(est_alpha.reshape((PY,PX)))
    full_res_depth.append(est_depth.reshape((PY,PX)))
    full_res_color.append(est_color.reshape((PY,PX,3)))

    print('.',end='')

In [ ]:
if os.path.exists(output_folder):
    import shutil
    shutil.rmtree(output_folder)
os.makedirs(output_folder, exist_ok=True)

In [ ]:
vecr = np.hstack([_.ravel() for _ in full_res_depth])
vecr = vecr[~np.isnan(vecr)]
vmin = np.percentile(vecr,5)
vmax = np.percentile(vecr,95)
vscale = vmax-vmin

In [ ]:
import matplotlib
from PIL import Image, ImageDraw, ImageFont
start_f = 0
avg_size = np.array([PX,PY])
fsize = irc(96/4)

font = ImageFont.truetype('Roboto-Regular.ttf', size=irc(avg_size[0]/8))
cmap = matplotlib.cm.get_cmap('viridis')
cmap2 = matplotlib.cm.get_cmap('magma')

for i,mask_res in enumerate(full_res_alpha):
    r_idx = frame_idxs[i]
    #img1 = ground_images[r_idx]/255.0*np.clip(full_masks[r_idx] > .1,0.3,1)[:,:,None]
    #img2 = ground_images[r_idx]*np.clip((mask_res)**0.4,0.05,1)[:,:,None]
    img2 = full_res_color[i]#np.tile(mask_res[:,:,None],(1,1,3))
    img_gt_mask = np.tile(masks[r_idx][:,:,None],(1,1,3))

    true_alpha = masks[r_idx]

    est_alpha = jnp.clip(mask_res,1e-6,1-1e-6)
    mask_loss = - ((true_alpha * jnp.log(est_alpha)) + (1-true_alpha)*jnp.log(1-est_alpha))
    loss_viz = cmap2(0.25*mask_loss)[:,:,:3]

    depth = cmap((full_res_depth[i]-vmin)/vscale)[:,:,:3]
    img2 = np.concatenate((images[r_idx]/255.0,img_gt_mask,loss_viz,img2, depth), axis=1)
    int_img = np.round(img2*255).astype(np.uint8)
    pil_img = Image.fromarray(int_img)
    d1 = ImageDraw.Draw(pil_img)
    d1.text((avg_size[0]*1.1, irc(fsize*0.1)), "Iteration: {:3d}\nEpoch: {:.1f}".format(opt_to_use[i],opt_to_use[i]/Niter_epoch), ha='center',font=font,fill=(180, 180, 180))
    d1.text((avg_size[0]*1.3, irc(avg_size[1]-fsize*1.5)), "Target Mask", font=font,fill=(255, 255, 255),ha='center')
    d1.text((avg_size[0]*2.4, irc(avg_size[1]-fsize*1.5)), "Loss", font=font,fill=(255, 255, 255),ha='center',align='center')
    d1.text((avg_size[0]*3.3, irc(avg_size[1]-fsize*2.5)), "Estimated\nColor", font=font,fill=(255, 255, 255),ha='center',align='center')
    d1.text((avg_size[0]*4.3, irc(avg_size[1]-fsize*2.5)), "Estimated\nDepth", font=font,fill=(255, 255, 255),ha='center',align='center')

    img3 = np.array(pil_img)
    
    sio.imsave('{}/{:03d}.jpg'.format(output_folder,i),img3,quality=95)


In [ ]:
(avg_size[0]*1.3, irc(avg_size[1]-fsize*1.5)),avg_size

In [ ]:
plt.figure(figsize=(18,8))
plt.imshow(img3)
plt.axis('off')

In [ ]:
import subprocess
if os.path.exists('{}.mp4'.format(output_folder)):
    os.remove('{}.mp4'.format(output_folder))
subprocess.call(' '.join(['/usr/bin/ffmpeg',
                 '-framerate','60',
                 '-i','{}/%03d.jpg'.format(output_folder),
                 '-vf','\"pad=ceil(iw/2)*2:ceil(ih/2)*2\"',
                 '-c:v','h264',
                 '-pix_fmt','yuv420p',
                 '{}.mp4'.format(output_folder)]),shell=True)